# Maximize macroscopic entropy production rate for the ABC Model

![ABC-model](Metabolic-network.JPG "ABC Metabolic network")

## All Constraints

$$ \begin{array}{ll}
    \underset{\vec{\bf r}_+,\vec{\bf r}_-,\vec{\bf c}}{\mbox{maximize}}   & - \sum_j\left({\mathscr P}_{+j}\log({\mathscr P}_{+j}) + {\mathscr P}_{-j}\log({\mathscr P}_{-j})\right)\cdot v_{growth}   \\
    \mbox{subject to}  & S\cdot(\vec{\bf L}_+-\vec{\bf L}_-) = 0  & \text{Steady-state Net Likelihood Constraint} \\
                         & S\cdot(\vec{\bf r}_+-\vec{\bf r}_-) = 0 & \text{Steady-state net flux Constraint} \\
                       &\log\vec{\bf r}_+ -\log\vec{\bf r}_-= -\frac{1}{RT}S^T\cdot\vec{\mu}^0 - S^T\cdot\log\vec{\bf c} & \text{Thermodynamic Constraint} \\
                      %% & \log\vec{\bf L}_- = \frac{1}{RT}S^T\cdot\vec{\mu}^0 + S^T\cdot\log\vec{\bf c} \\
                      %% & \log\vec{\bf L}_+  -\log\vec{\bf L}_- \\
                     %% & \vec{v}_{lower}(L_+) \leq \vec{\bf r}_+ -\vec{\bf r}_- \leq \vec{v}_{upper}(L_+) \\
                         & \vec{\bf r}_+ \geq 0 & \text{Positive rate Constraint} \\
                         & \vec{\bf r}_- \geq 0 & \text{Positive rate Constraint} \\
                      & \log \left[{\bf A}_{ext}\right] = \log \left[{\bf E}_{ext}\right] =  \log c_{upper} & \text{Input boundary Constraint} \\
                      & \log \left[{\bf E}_{ext}\right] = \log \left[{\bf D}_{ext}\right] = \log c_{lower}  & \text{Output boundary Constraint} \\         
      & -\left|\Delta G_{ext}\right| \leq S^T\mu \leq \left|\Delta G_{ext}\right|
    \end{array} 
$$

Where
* $\vec{\bf r}_+,\vec{\bf r}_-$ are decision variables representing the forward and backward rates, respectively.
* ${\mathscr P}_{+i},{\mathscr P}_{-i}$ are the normalized forward and backward thermodynamic driving forces $\frac{r_{+i}}{r_{-i}}\left(\sum_j\frac{r_{+j}}{r_{-j}} + \frac{r_{-j}}{r_{+j}}\right)^{-1}$ and $\frac{r_{-i}}{r_{+i}}\left(\sum_j\frac{r_{+j}}{r_{-j}} + \frac{r_{-j}}{r_{+j}}\right)^{-1}$
* $\vec{\bf c}$ is a decision variable representing the chemical concentrations of each metabolite 
*  $S$ is the $m\times n$ stoichiometric matrix of representing $m$ metabolites and $n$ reactions of the model  
*  $\vec{\mu}^0$ is the vector of standard chemical potentials 
* $L_+ = \frac{r_+}{r_-}$
* $L_- = \frac{r_-}{r_+}$
* $v_{lower}(L) =\min\left(-1,\text{sign}(\log L)\cdot L^{\text{sign}(\log L)}\right) + 1$
* $v_{upper}(L) = \max\left(1,\text{sign}(\log L)\cdot L^{\text{sign}(\log L)}\right) - 1$
* $\Delta G_{ext} =  \mu_{D_{ext}} + \mu_{F_{ext}} - \mu_{A_{ext}} - \mu_{E_{ext}}$

Decision variables are emphasized in **bold**. 

## Using Likelihoods and $\log$ concentrations as decision variables with PyOpt

In [1]:
import mentos
import pandas as pd, os, sys
import numpy as np
from mentos.abc_model import fullS, S, deltaG0, mu0, \
     c_U, c_L, mets, external_mets, met_bounds, internal_mets, rxns, \
     efflux, uptake, n, m, R, T, biomass, \
     external_free_energy, efflux, uptake, A_ext, D_ext, E_ext, F_ext, \
     v_L, v_U
from scipy.stats import entropy
from IPython.display import display, Markdown, Latex, SVG, Image, HTML
abc_dir = ''
metab, reactions = {}, {}

convex_mentos = 'convex_mentos_likelihoods_first_elmo'
metab[convex_mentos] = pd.read_table(os.path.join(convex_mentos, 'ABC_metab_{}.csv'.format(convex_mentos)), index_col=0)
#display(metab[convex_mentos])

boltzmann = 'boltzmann'
metab[boltzmann] = pd.read_table(os.path.join(boltzmann, 'ABC_metab_{}.csv'.format(boltzmann)), index_col=0)
#display(metab[convex_mentos])

reactions[convex_mentos] = pd.read_table(os.path.join(convex_mentos, 'ABC_reaction_{}.csv'.format(convex_mentos)), index_col=0)
reactions[boltzmann] = pd.read_table(os.path.join(boltzmann, 'ABC_reaction_{}.csv'.format(boltzmann)), index_col=0)
#display(reactions[convex_mentos])

m,n = fullS.shape
i,n = S.shape

def macroscopic_entropy_production_rate_and_net_likelihood_ss( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
    return entropy( np.concatenate( ( forward_probability, backward_probability ) ) )*net_flux[biomass] - np.sum(np.square(np.dot(S,forward_likelihood - backward_likelihood)))

def constraints_on_likelihoods( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
    return np.concatenate((np.log(forward_likelihood)  + log_Q - log_K, # = 0
                           [  log_c[A_ext] - np.log(c_U),     # = 0               # Creating a concentration gradient between A, E and D,F
                              log_c[E_ext] - np.log(c_U),     # = 0
                              log_c[D_ext] - np.log(c_L),     # = 0
                              log_c[F_ext] - np.log(c_L) ]))  #= 0 

def macroscopic_entropy_production_rate( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables( x, fullS, mu0, deltaG0, R, T )
    return entropy( np.concatenate( ( forward_probability, backward_probability ) ) )*net_flux[biomass]

def all_constraints( x, fullS, mu0, deltaG0, R, T  ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables( x, fullS, mu0, deltaG0, R, T )
    return np.concatenate((np.dot(S,forward_likelihood - backward_likelihood),
                            np.dot(S,net_flux),
                            np.log(forward_rate) - np.log(backward_rate) + log_Q - log_K,
                            [ log_c[A_ext] - np.log(c_U),  # = 0               # Creating a concentration gradient between A, E and D,F
                              log_c[E_ext] - np.log(c_U),  # = 0
                              log_c[D_ext] - np.log(c_L),  # = 0
                              log_c[F_ext] - np.log(c_L)],  #= 0
                            -abs(external_free_energy) - np.dot(fullS.T.as_matrix(),mu), # <= 0
                            np.dot(fullS.T.as_matrix(),mu) - abs(external_free_energy),  # <= 0 
                            np.minimum(-1, thermodynamic_driving_force)+ 1 -net_flux, # <= 0
                            net_flux - np.maximum(1, thermodynamic_driving_force ) - 1)
                           ) # <= 0

def thermo_and_boundary( x, fullS, mu0, deltaG0, R, T ):
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux = mentos.make_variables( x, fullS, mu0, deltaG0, R, T )
    return np.concatenate((np.log(forward_rate) - np.log(backward_rate) + log_Q - log_K,
                           [  log_c[A_ext] - np.log(c_U),  # = 0               # Creating a concentration gradient between A, E and D,F
                              log_c[E_ext] - np.log(c_U),  # = 0
                              log_c[D_ext] - np.log(c_L),  # = 0
                              log_c[F_ext] - np.log(c_L)]))  #= 0)
def make_macro_maxent_production_rate_objective(fullS, S, mu0, deltaG0, R, T, c_L, c_U,v_L, v_U, objective_function, constraint_function):
    external_mets = [met for met in fullS.index if 'ext' in met]
    met_bounds = pd.Series({'A_ext':c_U, 'E_ext': c_U, 'F_ext': c_L, 'D_ext': c_L}, index=external_mets)
    mu_ext = mu0[external_mets] + R*T*met_bounds.apply(np.log)
    external_free_energy =  (mu_ext['D_ext'] + mu_ext['F_ext']) - (mu_ext['A_ext'] + mu_ext['E_ext'])
    
    def macro_maxent_production_rate(x):
        """Maximize product of  entropy production and macroscopic biomass growth rate of the ABC model"""
        f = objective_function( x, fullS, mu0, deltaG0, R, T  )
        g = constraint_function( x, fullS, mu0, deltaG0, R, T )
        fail = 0
        return -f, g, fail
    return macro_maxent_production_rate

In [ ]:

variable_group = {'rates_and_log_c':[  
                    dict(name='log_c',nvars=m, type='c', initial_value=metab[convex_mentos]['Log Concentrations'].values),
                    dict(name='forward_rate', nvars=n, type='c', initial_value=reactions[convex_mentos]['Forward rate'].values),
                    dict(name='backward_rate',nvars=n, type='c', initial_value=reactions[convex_mentos]['Backward rate'].values)],
                 'likelihoods_and_log_c': [
                    dict(name='forward_likelihood', nvars=n, type='c', initial_value=reactions[convex_mentos]['Forward likelihoods'].values),
                    dict(name='log_c',nvars=m, type='c', initial_value=metab[convex_mentos]['Log Concentrations'].values)]}
       
constraint_group={'all_constraints':[
                    dict(name='steady_state_net_likelihood', ncons=i, type='e'),
                    dict(name='steady_state_flux', ncons=i, type='e'),
                    dict(name='thermodynamics',ncons=n, type='e'),
                    dict(name='boundary_conditions', ncons=m-i, type='e'),
                    dict(name='energy_barrier', ncons=n, type='i'),
                    dict(name='energy_sink',ncons=n,type='i'),
                    dict(name='flux_lower_bounds', ncons=n, type='i'),
                    dict(name='flux_upper_bounds', ncons=n,type='i')],
                  'thermo_and_boundary':[dict(name='thermodynamics',ncons=n, type='e'),
                                         dict(name='boundary_conditions', ncons=m-i, type='e'),],
                  'constraints_on_likelihoods': [dict(name='thermodynamics',ncons=n, type='e'),
                                                  dict(name='boundary_conditions', ncons=m-i, type='e'),]}
constraint_function = {'all_constraints': all_constraints,
                      'thermo_and_boundary': thermo_and_boundary,
                       'constraints_on_likelihoods': constraints_on_likelihoods
                      }
objective_function = {'macroscopic_entropy_production_rate': macroscopic_entropy_production_rate,
                      'macroscopic_entropy_production_rate_and_net_likelihood_ss':macroscopic_entropy_production_rate_and_net_likelihood_ss}

algorithms = [#('macroscopic_entropy_production_rate','rates_and_log_c', 'all_constraints' ), 
              #('macroscopic_entropy_production_rate','rates_and_log_c', 'thermo_and_boundary'), 
              ('macroscopic_entropy_production_rate_and_net_likelihood_ss','likelihoods_and_log_c', 'constraints_on_likelihoods')]

for og, vg, cg in algorithms:
    metab[cg], reactions[cg] = mentos.run_mentos_w_likelihoods(fullS, S, internal_mets, deltaG0, mu0, 
                                        variable_group[vg], constraint_group[cg],
                                        obj=make_macro_maxent_production_rate_objective( fullS, S, mu0, deltaG0, R, T, c_L, c_U,v_L, v_U, 
                                                                                         objective_function=objective_function[og],
                                                                                         constraint_function=constraint_function[cg]),
                                        biomass_rxn='R_8')
    mentos.print_report( os.path.join(abc_dir, cg), 'ABC_metab_{}.csv', metab[cg] )
    mentos.print_report( os.path.join(abc_dir, cg), 'ABC_reaction_{}.csv', reactions[cg])
    #display(Markdown('### ' + cg))
    #display(metab[cg].T)
                             
    

#display(HTML(mentos.compare_frames(**metab).to_html()))

#display(HTML(mentos.compare_frames(**reactions).to_html()))

## Using Likelihoods and $\log$ concentrations as decision variables with SciPy

In [2]:
from mentos.abc_model import fullS, S, deltaG0, mu0, \
     c_U, c_L, mets, external_mets, met_bounds, internal_mets, rxns, \
     efflux, uptake, n, m, R, T, biomass, \
     external_free_energy, efflux, uptake, A_ext, D_ext, E_ext, F_ext, \
     v_L, v_U
import mentos
import scipy
from scipy.stats import entropy
from IPython.display import Latex, Markdown, display

obj = 'macro_maxent_production_rate_ss_net_likelihood_scipy_w_convex_ic'
initial_value = 'convex_mentos_likelihoods_first_elmo'

#modeldir = 'Boltzmann-2017-09-09/Models/ABC_model/'
#abc_model = cobra.io.read_json_model(os.path.join(modeldir,'abc_model.json'))
#noext = [r.id for r in core_model.reactions if 'EX_' not in r.id]

#fullS = pd.DataFrame(cobra.util.array.create_stoichiometric_matrix(core_model), 
#                    index=[m.id for m in core_model.metabolites],
#                    columns = [r.id for r in core_model.reactions])[noext]
mets, rxns = fullS.index, fullS.columns
m,n = fullS.shape
internal_mets = [m for m in mets if '_e' not in m]
#display(HTML(fullS.to_html()))
m,n = fullS.shape
rxns, mets = fullS.columns, fullS.index
S = fullS.loc[internal_mets].as_matrix()
i,n = S.shape
biomass_rxn =  "R_8"
biomass = rxns.get_loc(biomass_rxn)
#met_bounds = getMetBounds(fullS, reactions[initial_value]['Net flux'], c_L, c_U, m9_medium, ext=r'_ext')
#external_free_energy = fullS.T[external_mets].dot(mu_ext).sum() #dot(fluxes)
met_lb = np.log(1e-8)#set_log_conc( met_bounds, set_log_conc(  ecoli_comp_abs_conc.loc[fitted_mets,'L.B.'], pd.Series(np.log(c_L),index=mets) )).values
met_ub = np.log(1e-2)# set_log_conc( met_bounds, set_log_conc(  ecoli_comp_abs_conc.loc[fitted_mets,'U.B.'], pd.Series(np.log(c_U),index=mets ))).values

#rxnbounds = get_rxn_bounds( core_model )
#rxnbounds.loc['GLCpts', 'upper'] = 1

#lower,upper = rxnbounds.loc[rxns, 'lower'].values, rxnbounds.loc[rxns,'upper'].astype(float).values


def make_macro_maxent_production_rate_objective(initial_log_c, initial_forward_likelihood, fullS, S, mu0, deltaG0, R, T, log_c_L, log_c_U,v_L, v_U, external_free_energy, met_bounds,biomass):

    def check_constraints( x0, constraints,rxns, mets ):
        internal_mets = [m for m in mets if '_e' not in m]
        for constraint in constraints: 
            constraint_eval = constraint['fun'](x0)
            if constraint['type'] == 'eq':
                if np.allclose(constraint_eval, 0):
                    display(Latex('{}'.format(constraint['fun'].__doc__)))
                    if 'jac' in constraint:
                        display(Latex('{}'.format(constraint['jac'].__doc__)))
                else:
                    display('{} violated'.format(constraint['fun'].__name__))
                    if len(constraint_eval) == len(internal_mets):
                        constraint_s = pd.Series(constraint_eval,index=internal_mets)
                        
                        display(constraint_s[~constraint_s.apply(lambda x: np.isclose(x,0))].to_frame(constraint['fun'].__doc__))
                    elif len(constraint_eval) == len(rxns):
                        display(pd.DataFrame(constraint_eval, index=rxns))
                    
            elif constraint['type'] == 'ineq':
                if np.all(constraint_eval >= 0):
                    display(Latex('{}'.format(constraint['fun'].__doc__)))
                else:
                    display('{} violated'.format(constraint['fun'].__name__))
                    if len(constraint_eval) == len(rxns):
                        display(pd.DataFrame(constraint_eval,index=rxns))
                
            
    def make_logc_bounds(mets, met_bounds,epsilon=1):
        logc_bounds = []
        for met in mets:
            if met in met_bounds:
                logc_bounds.append((np.log(met_bounds[met]), np.log(met_bounds[met])))
            else:
                logc_bounds.append((log_c_L, log_c_U))
        return logc_bounds
    
    def make_rate_bounds( rxns, r_L=0, r_U=None):
        return [(r_L, r_U) for rxn in rxns]
                
    def macro_entropy_production_rate(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy(np.concatenate( (forward_probability, backward_probability ) ) )*net_flux[biomass] #- 1000*np.sum(np.abs(slack))
        return -f
    
    def macro_entropy_production_rate_and_net_likelihood_ss(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy(np.concatenate( (forward_probability, backward_probability ) ) )*net_flux[biomass] - 1000*np.sum(np.square(np.dot(S, forward_likelihood - backward_likelihood)))
        return -f
        
    def macro_entropy_production_rate_thermo_and_net_likelihood_ss(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy( np.concatenate(( forward_probability, backward_probability ) ) )*net_flux[biomass] \
            - 1000*np.sum( np.square( np.dot( S, forward_likelihood - backward_likelihood ))) \
            - 1000*np.sum( np.square( np.log( forward_likelihood ) - log_K + log_Q ))
        return -f
        
    def macro_entropy_production_rate_and_thermo(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy(np.concatenate( (forward_probability, backward_probability ) ) )*net_flux[biomass] - 1000*np.sum(np.square(np.log(forward_likelihood) - log_K + log_Q))
        return -f
    def steady_state_net_likelihood_constraint(x):
        """$$S\cdot L_{net} = 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.dot(S,forward_likelihood-backward_likelihood)
    def steady_state_constraint(x):
        """$$S\cdot v = 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.dot(S,net_flux)
    def thermodynamic_constraint(x):
        """$$\log L_+ + \log Q - \log K = 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.log(forward_likelihood) + log_Q - log_K
    def energy_barrier_constraint( x ):
        """$$ \|\Delta G_{ext}\| - S^T\mu \geq 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return   np.abs(external_free_energy) - np.dot(fullS.T.as_matrix(),mu)
    def energy_sink_constraint( x ):
        """$$S^T\mu + \|\Delta G_{ext}\| \geq 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return  np.dot(fullS.T.as_matrix(),mu) + np.abs(external_free_energy) 
    def second_law_constraint( x ):
        """$$\log L_+ \geq 0 \iff v \geq 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux= mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.log(forward_likelihood)*net_flux
    def flux_upper_constraint( x ):
        """$$ v_U - r_+ + r_- \geq 0 $$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux= mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return v_U - forward_rate + backward_rate
    def flux_lower_constraint( x ):
        """$$ r_+ - r_- - v_L \geq 0  $$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux= mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return  forward_rate - backward_rate - v_L
    x0 = np.concatenate((initial_log_c.values,
                         initial_forward_likelihood.values))
    constraints = [#dict(type='eq',  fun=steady_state_net_likelihood_constraint),
                   #dict(type='eq',  fun=steady_state_constraint),
                   #dict(type='eq',  fun=thermodynamic_constraint),
                   #dict(type='ineq',fun=energy_barrier_constraint),
                   #dict(type='ineq',fun=energy_sink_constraint),
                   #dict(type='ineq',fun=second_law_constraint),
                   #dict(type='ineq',fun=flux_upper_constraint),
                   #dict(type='ineq',fun=flux_lower_constraint)
]
                
    bounds = np.concatenate((make_logc_bounds(mets, met_bounds),
                             make_rate_bounds( rxns )))
    check_constraints(x0, constraints, rxns, mets)
    mentos_result = scipy.optimize.minimize( fun=macro_entropy_production_rate_thermo_and_net_likelihood_ss,
                                            x0=x0,
                                            method='SLSQP',
                                            bounds=bounds,
                                            constraints=constraints,
                                            options=dict(disp=True))
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux= mentos.make_variables_from_likelihoods( mentos_result.x, fullS, mu0, deltaG0, R, T )
    
    return mentos.generate_metabolite_report(log_c, forward_rate, backward_rate, S, mets, internal_mets,rxns, fullS, mu0), \
           mentos.generate_rxn_report(mets, log_c, log_Q, log_K,forward_rate, 
                                                backward_rate, rxns, deltaG0, biomass_rxn), mentos_result
metab[obj], reactions[obj], mentos_result = make_macro_maxent_production_rate_objective( 
                                metab[initial_value]['Log Concentrations'],
                                reactions[initial_value]['Forward likelihoods'],
                                fullS, S, mu0[mets], deltaG0[rxns], R, T, 
                                met_lb, met_ub, v_L, v_U, external_free_energy, met_bounds, biomass)

mentos.print_report( obj, 'ABC_metab_{}.csv', metab[obj] )
mentos.print_report( obj, 'ABC_reaction_{}.csv', reactions[obj])
  
if mentos_result.success:
        print("Success!")
else:
        print(mentos_result.message)
        print(mentos_result.keys())

algorithm = obj


/Users/zuck016/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/mentos/mentos.py:247 RuntimeWarning: divide by zero encountered in reciprocal
/Users/zuck016/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/mentos/mentos.py:252 RuntimeWarning: invalid value encountered in multiply
/Users/zuck016/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/mentos/mentos.py:256 RuntimeWarning: invalid value encountered in divide
/Users/zuck016/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/mentos/mentos.py:257 RuntimeWarning: divide by zero encountered in log
/Users/zuck016/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/mentos/mentos.py:258 RuntimeWarning: divide by zero encountered in power
/Users/zuck016/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:94 RuntimeWarning: divide by zero encountered in log


Iteration limit exceeded    (Exit mode 9)
            Current function value: 27975.4625967
            Iterations: 101
            Function evaluations: 2249
            Gradient evaluations: 98
Iteration limit exceeded
['status', 'success', 'njev', 'nfev', 'fun', 'x', 'message', 'jac', 'nit']


In [3]:
from mentos.abc_model import fullS, S, deltaG0, mu0, \
     c_U, c_L, mets, external_mets, met_bounds, internal_mets, rxns, \
     efflux, uptake, n, m, R, T, biomass, \
     external_free_energy, efflux, uptake, A_ext, D_ext, E_ext, F_ext, \
     v_L, v_U
import mentos
import scipy
from scipy.stats import entropy
from IPython.display import Latex, Markdown, display

obj = 'macro_maxent_production_rate_ss_net_likelihood_scipy_w_boltzman_ic'
initial_value = 'boltzmann'

#modeldir = 'Boltzmann-2017-09-09/Models/ABC_model/'
#abc_model = cobra.io.read_json_model(os.path.join(modeldir,'abc_model.json'))
#noext = [r.id for r in core_model.reactions if 'EX_' not in r.id]

#fullS = pd.DataFrame(cobra.util.array.create_stoichiometric_matrix(core_model), 
#                    index=[m.id for m in core_model.metabolites],
#                    columns = [r.id for r in core_model.reactions])[noext]
mets, rxns = fullS.index, fullS.columns
m,n = fullS.shape
internal_mets = [m for m in mets if '_e' not in m]
#display(HTML(fullS.to_html()))
m,n = fullS.shape
rxns, mets = fullS.columns, fullS.index
S = fullS.loc[internal_mets].as_matrix()
i,n = S.shape
biomass_rxn =  "R_8"
biomass = rxns.get_loc(biomass_rxn)
#met_bounds = getMetBounds(fullS, reactions[initial_value]['Net flux'], c_L, c_U, m9_medium, ext=r'_ext')
#external_free_energy = fullS.T[external_mets].dot(mu_ext).sum() #dot(fluxes)
met_lb = np.log(1e-8)#set_log_conc( met_bounds, set_log_conc(  ecoli_comp_abs_conc.loc[fitted_mets,'L.B.'], pd.Series(np.log(c_L),index=mets) )).values
met_ub = np.log(1e-2)# set_log_conc( met_bounds, set_log_conc(  ecoli_comp_abs_conc.loc[fitted_mets,'U.B.'], pd.Series(np.log(c_U),index=mets ))).values

#rxnbounds = get_rxn_bounds( core_model )
#rxnbounds.loc['GLCpts', 'upper'] = 1

#lower,upper = rxnbounds.loc[rxns, 'lower'].values, rxnbounds.loc[rxns,'upper'].astype(float).values


def make_macro_maxent_production_rate_objective(initial_log_c, initial_forward_likelihood, fullS, S, mu0, deltaG0, R, T, log_c_L, log_c_U,v_L, v_U, external_free_energy, met_bounds,biomass):

    def check_constraints( x0, constraints,rxns, mets ):
        internal_mets = [m for m in mets if '_e' not in m]
        for constraint in constraints: 
            constraint_eval = constraint['fun'](x0)
            if constraint['type'] == 'eq':
                if np.allclose(constraint_eval, 0):
                    display(Latex('{}'.format(constraint['fun'].__doc__)))
                    if 'jac' in constraint:
                        display(Latex('{}'.format(constraint['jac'].__doc__)))
                else:
                    display('{} violated'.format(constraint['fun'].__name__))
                    if len(constraint_eval) == len(internal_mets):
                        constraint_s = pd.Series(constraint_eval,index=internal_mets)
                        
                        display(constraint_s[~constraint_s.apply(lambda x: np.isclose(x,0))].to_frame(constraint['fun'].__doc__))
                    elif len(constraint_eval) == len(rxns):
                        display(pd.DataFrame(constraint_eval, index=rxns))
                    
            elif constraint['type'] == 'ineq':
                if np.all(constraint_eval >= 0):
                    display(Latex('{}'.format(constraint['fun'].__doc__)))
                else:
                    display('{} violated'.format(constraint['fun'].__name__))
                    if len(constraint_eval) == len(rxns):
                        display(pd.DataFrame(constraint_eval,index=rxns))
                
            
    def make_logc_bounds(mets, met_bounds,epsilon=1):
        logc_bounds = []
        for met in mets:
            if met in met_bounds:
                logc_bounds.append((np.log(met_bounds[met]), np.log(met_bounds[met])))
            else:
                logc_bounds.append((log_c_L, log_c_U))
        return logc_bounds
    
    def make_rate_bounds( rxns, r_L=0, r_U=None):
        return [(r_L, r_U) for rxn in rxns]
                
    def macro_entropy_production_rate(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy(np.concatenate( (forward_probability, backward_probability ) ) )*net_flux[biomass] #- 1000*np.sum(np.abs(slack))
        return -f
    
    def macro_entropy_production_rate_and_net_likelihood_ss(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy(np.concatenate( (forward_probability, backward_probability ) ) )*net_flux[biomass] - 1000*np.sum(np.square(np.dot(S, forward_likelihood - backward_likelihood)))
        return -f
        
    def macro_entropy_production_rate_thermo_and_net_likelihood_ss(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy( np.concatenate(( forward_probability, backward_probability ) ) )*net_flux[biomass] \
            - 1000*np.sum( np.square( np.dot( S, forward_likelihood - backward_likelihood ))) \
            - 1000*np.sum( np.square( np.log( forward_likelihood ) - log_K + log_Q ))
        return -f
        
    def macro_entropy_production_rate_and_thermo(x):
        """Maximize product of  entropy production difference and macroscopic biomass growth rate of the ABC model"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        f = entropy(np.concatenate( (forward_probability, backward_probability ) ) )*net_flux[biomass] - 1000*np.sum(np.square(np.log(forward_likelihood) - log_K + log_Q))
        return -f
    def steady_state_net_likelihood_constraint(x):
        """$$S\cdot L_{net} = 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.dot(S,forward_likelihood-backward_likelihood)
    def steady_state_constraint(x):
        """$$S\cdot v = 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.dot(S,net_flux)
    def thermodynamic_constraint(x):
        """$$\log L_+ + \log Q - \log K = 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.log(forward_likelihood) + log_Q - log_K
    def energy_barrier_constraint( x ):
        """$$ \|\Delta G_{ext}\| - S^T\mu \geq 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return   np.abs(external_free_energy) - np.dot(fullS.T.as_matrix(),mu)
    def energy_sink_constraint( x ):
        """$$S^T\mu + \|\Delta G_{ext}\| \geq 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux = mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return  np.dot(fullS.T.as_matrix(),mu) + np.abs(external_free_energy) 
    def second_law_constraint( x ):
        """$$\log L_+ \geq 0 \iff v \geq 0$$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux= mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return np.log(forward_likelihood)*net_flux
    def flux_upper_constraint( x ):
        """$$ v_U - r_+ + r_- \geq 0 $$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux= mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return v_U - forward_rate + backward_rate
    def flux_lower_constraint( x ):
        """$$ r_+ - r_- - v_L \geq 0  $$"""
        log_c, \
        forward_rate, backward_rate, \
        log_Q, log_K, \
        forward_likelihood, backward_likelihood, \
        forward_probability, backward_probability, mu, \
        thermodynamic_driving_force, \
        net_flux= mentos.make_variables_from_likelihoods( x, fullS, mu0, deltaG0, R, T )
        return  forward_rate - backward_rate - v_L
    x0 = np.concatenate((initial_log_c.values,
                         initial_forward_likelihood.values))
    constraints = [#dict(type='eq',  fun=steady_state_net_likelihood_constraint),
                   #dict(type='eq',  fun=steady_state_constraint),
                   #dict(type='eq',  fun=thermodynamic_constraint),
                   #dict(type='ineq',fun=energy_barrier_constraint),
                   #dict(type='ineq',fun=energy_sink_constraint),
                   #dict(type='ineq',fun=second_law_constraint),
                   #dict(type='ineq',fun=flux_upper_constraint),
                   #dict(type='ineq',fun=flux_lower_constraint)
]
                
    bounds = np.concatenate((make_logc_bounds(mets, met_bounds),
                             make_rate_bounds( rxns )))
    check_constraints(x0, constraints, rxns, mets)
    mentos_result = scipy.optimize.minimize( fun=macro_entropy_production_rate_thermo_and_net_likelihood_ss,
                                            x0=x0,
                                            method='SLSQP',
                                            bounds=bounds,
                                            constraints=constraints,
                                            options=dict(disp=True))
    log_c, \
    forward_rate, backward_rate, \
    log_Q, log_K, \
    forward_likelihood, backward_likelihood, \
    forward_probability, backward_probability, mu, \
    thermodynamic_driving_force, \
    net_flux= mentos.make_variables_from_likelihoods( mentos_result.x, fullS, mu0, deltaG0, R, T )
    
    return mentos.generate_metabolite_report(log_c, forward_rate, backward_rate, S, mets, internal_mets,rxns, fullS, mu0), \
           mentos.generate_rxn_report(mets, log_c, log_Q, log_K,forward_rate, 
                                                backward_rate, rxns, deltaG0, biomass_rxn), mentos_result
metab[obj], reactions[obj], mentos_result = make_macro_maxent_production_rate_objective( 
                                metab[initial_value]['Log Concentrations'],
                                reactions[initial_value]['Forward likelihoods'],
                                fullS, S, mu0[mets], deltaG0[rxns], R, T, 
                                met_lb, met_ub, v_L, v_U, external_free_energy, met_bounds, biomass)

mentos.print_report( obj, 'ABC_metab_{}.csv', metab[obj] )
mentos.print_report( obj, 'ABC_reaction_{}.csv', reactions[obj])
  
if mentos_result.success:
        print("Success!")
else:
        print(mentos_result.message)
        print(mentos_result.keys())

algorithm = obj


/Users/zuck016/.pyenv/versions/anaconda2-4.4.0/lib/python2.7/site-packages/ipykernel_launcher.py:94 RuntimeWarning: divide by zero encountered in log


Optimization terminated successfully.    (Exit mode 0)
            Current function value: -202.345005518
            Iterations: 23
            Function evaluations: 524
            Gradient evaluations: 23
Success!


In [4]:
display(HTML(mentos.compare_frames(**metab).to_html()))

display(HTML(mentos.compare_frames(**reactions).to_html()))
